In [ ]:
import pandas as pd

file_path = 'data/Supplemental_Data_Raw_Genecounts.csv'

# Load the CSV file into a DataFrame.
df = pd.read_csv(file_path)
df = df.drop('Unnamed: 17', axis=1)
df = df.rename(columns={"Unnamed: 0": "mnra",})
df.set_index('mnra', inplace=True)
# Convert every non-index column to int
df = df.apply(pd.to_numeric, errors='coerce', downcast='integer')

# Pruning rows where all the values are 0.
# Assuming that you have columns 'x' and 'y', replace them with the actual names of your columns.
print(df.shape)
df = df[(df.loc[:, df.columns != 'index'] != 0).any(axis=1)]


In [ ]:
columns_to_check = df.columns.difference(['Unnamed: 0'])

# Pruning rows where all the values in columns_to_check are 0.
df = df[(df[columns_to_check] != 0).any(axis=1)]


In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
import matplotlib.pyplot as plt

# Calculating the mean and standard deviation for each row
df['mean'] = df[columns_to_check].mean(axis=1)
df['std'] = df[columns_to_check].std(axis=1)
df['cv'] = (df.std(axis=1) / df.mean(axis=1)).abs() * 100

import numpy as np

# Generate a color map
colors = plt.cm.viridis(np.linspace(0, 1, len(df)))

plt.figure(figsize=(10, 6))
for idx, color in zip(df.index, colors):
    plt.errorbar(idx, df.loc[idx, 'mean'], yerr=df.loc[idx, 'std'], fmt='o', color=color)

plt.title('Mean and Variability of Each Sample')
plt.xlabel('Sample Index')
plt.ylabel('Mean')
plt.grid(True)
plt.show()

In [ ]:
# Plotting histogram for df['cv']
plt.figure(figsize=(10,6))
plt.hist(df['cv'], bins=200, color='blue', edgecolor='black')
plt.title('Histogram of Coefficient of Variation (CV)')
plt.xlabel('CV')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

# Plotting histogram for df['mean']
plt.figure(figsize=(10,6))
plt.hist(df['mean'], bins=200, color='green', edgecolor='black')
plt.title('Histogram of Mean')
plt.xlabel('Mean')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
import seaborn as sns

# Or, if you want to plot the raw counts or normalized counts, you can skip the correlation computation and directly plot the DataFrame
plt.figure(figsize=(10, 10))  # You can specify the figure size as per your requirement

n = 50  # You can set the number of genes you want to display
df_high_variance = df.loc[df.var(axis=1).nlargest(n).index]


# Plot the heatmap
sns.heatmap(df_high_variance, cmap='coolwarm', center=0, linewidths=.5)  # You can customize the colormap, center value, and linewidths

# Add title and labels if needed
plt.title('Heatmap of Gene Expressions')
plt.xlabel('Samples/Conditions')
plt.ylabel('Genes')

# Show the plot
plt.show()


## Differential expression analysis

In [ ]:
from statsmodels.stats import multitest
from scipy import stats
import itertools
from joblib import Parallel, delayed
from tqdm import tqdm

# Define the function to perform the t-test for a given pair of conditions and a given gene
def perform_ttest(condition_1, condition_2, gene_row):
    condition_1_data = gene_row[condition_1]
    condition_2_data = gene_row[condition_2]
    
    # If the variance is zero for both conditions, skip the t-test
    if np.var([condition_1_data]) == 0 and np.var([condition_2_data]) == 0:
        return {
            'condition_1': condition_1,
            'condition_2': condition_2,
            'gene': gene_row.name,
            'mean_condition_1': condition_1_data,
            'mean_condition_2': condition_2_data,
            'p_value': np.nan  # or return some default p_value, like 1.0
        }
    
    # Compute the mean expression level for each condition
    mean_condition_1 = np.mean(condition_1_data)
    mean_condition_2 = np.mean(condition_2_data)
    
    # Perform a t-test between the two conditions
    t_stat, p_value = stats.ttest_ind([condition_1_data], [condition_2_data], equal_var=False, nan_policy='omit')
    
    return {
        'condition_1': condition_1,
        'condition_2': condition_2,
        'gene': gene_row.name,
        'mean_condition_1': mean_condition_1,
        'mean_condition_2': mean_condition_2,
        'p_value': p_value
    }

# Get all condition columns
condition_columns = df.columns


# Create a list to store the tasks
tasks = []

# Initialize the progress bar
total_combinations = len(list(itertools.combinations(condition_columns, 2))) * len(df.index)
progress_bar = tqdm(total=total_combinations, desc='Generating tasks', unit='task')

# Generate all tasks (pairs of conditions and genes) with progress update
for condition_1, condition_2 in itertools.combinations(condition_columns, 2):
    for gene in df.index:
        tasks.append((condition_1, condition_2, df.loc[gene]))
        progress_bar.update(1)

# Close the progress bar
progress_bar.close()


In [ ]:

# Perform the tasks in parallel with progress bar
results_list = Parallel(n_jobs=-1)(delayed(perform_ttest)(condition_1, condition_2, gene_row) for condition_1, condition_2, gene_row in tqdm(tasks, desc='Processing', unit='task'))

# Convert the results list to a DataFrame
results_df = pd.DataFrame(results_list)

# Correct for multiple testing (optional)
results_df['adjusted_p_value'] = multitest.multipletests(results_df['p_value'], method='fdr_bh')[1]

# You can filter the results based on a significance threshold, e.g. adjusted p-value < 0.05
significant_results = results_df[results_df['adjusted_p_value'] < 0.05]


In [ ]:
results_df.columns

### Volcano plot

In [ ]:
from scipy import stats
import numpy as np

# Define the groups you want to compare
group_a_cols = [col for col in df.columns if 'A' in col]
group_b_cols = [col for col in df.columns if 'B' in col]

# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['gene', 'mean_group_a', 'mean_group_b', 'p_value'])

for index, row in df.iterrows():
    group_a_data = row[group_a_cols]
    group_b_data = row[group_b_cols]
    
    # Compute the mean expression level for each group
    mean_group_a = np.mean(group_a_data)
    mean_group_b = np.mean(group_b_data)
    
    # Perform a t-test between the two groups
    t_stat, p_value = stats.ttest_ind(group_a_data, group_b_data, equal_var=False, nan_policy='omit')
    
    # Append the results to the results DataFrame
    results_df = results_df.append({
        'gene': index,
        'mean_group_a': mean_group_a,
        'mean_group_b': mean_group_b,
        'p_value': p_value
    }, ignore_index=True)

# Correct for multiple testing (optional)
results_df['adjusted_p_value'] = multitest.multipletests(results_df['p_value'], method='fdr_bh')[1]

# Filter the results based on a significance threshold, for example, adjusted p-value < 0.05
significant_genes = results_df[results_df['adjusted_p_value'] < 0.05]

In [ ]:
significant_genes.shape

In [ ]:
from scipy import stats
import pandas as pd
import numpy as np
import statsmodels.stats.multitest as sm

# Assume df is your DataFrame with gene expression data

# Define the groups you want to compare
group_a_cols = [col for col in df.columns if 'A' in col]
group_b_cols = [col for col in df.columns if 'B' in col]

# Create an empty list to store DataFrames
results_dfs = []

for gene, row in df.iterrows():
    group_a_data = row[group_a_cols]
    group_b_data = row[group_b_cols]
    
    # Compute the mean expression level for each group
    mean_group_a = np.mean(group_a_data)
    mean_group_b = np.mean(group_b_data)
    
    # Perform a t-test between the two groups
    t_stat, p_value = stats.ttest_ind(group_a_data, group_b_data, equal_var=False, nan_policy='omit')
    
    # Create a DataFrame for each gene result
    gene_result_df = pd.DataFrame({'gene': [gene], 'mean_group_a': [mean_group_a], 'mean_group_b': [mean_group_b], 'p_value': [p_value]})
    
    # Append the DataFrame to the list
    results_dfs.append(gene_result_df)

# Concatenate all the DataFrames in the list into one DataFrame
results_df = pd.concat(results_dfs, ignore_index=True)

# Correct for multiple testing (optional)
results_df['adjusted_p_value'] = sm.multipletests(results_df['p_value'], method='fdr_bh')[1]

# Filter the results based on a significance threshold, for example, adjusted p-value < 0.05
significant_genes = results_df[results_df['adjusted_p_value'] < 0.05]